In [1]:
# Improt packages
import pandas as pd
import pyarrow
import json 

# Import functions from apps folder
from apps.collect_data import *

In [4]:
# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Women's_Prize_for_Fiction_winners"

books = get_wikidata(url)
books.head()


,Year,Author,Title,Result
0,2020,Maggie O'Farrell,Hamnet,Winner
1,2020,Angie Cruz,Dominicana,Finalist
2,2020,Bernardine Evaristo,"Girl, Woman, Other",Finalist
3,2020,Natalie Haynes,A Thousand Ships,Finalist
4,2020,Hilary Mantel,The Mirror and the Light,Finalist


In [5]:
books.to_parquet('assets/WPH.parquet')

## Info from book APIs

 Adding information from Google Book API and Open Library to the inital dataframe from the List of Womens Price for Fiction finalists and winners from the Wikipedapage 

In [6]:
# Collecting data from GBApi and OLApi
df = book_info_add(books)

In [5]:
# storing the dataframe as parquet in the assets folder
df.to_parquet('assets/GBA.parquet')

In [7]:
# Merge dataframes 
book_df = pd.merge(df,
                     books,
                     on='Title', how='outer')

In [8]:
# storing the dataframe as parquet in the assets folder
book_df.to_parquet('assets/book_df.parquet')

In [9]:
# Checking that all books where merged by seing if there are any observations from the right side (no value in the Author(s) column)
book_df.loc[book_df['Author(s)'].isnull()==True]


,Title,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language,Year,Author,Result


In [9]:
book_df

,Title,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language,Year,Author,Result
0,Hamnet,Maggie O'Farrell,2020-07-21,NATIONAL BOOK CRITICS CIRCLE AWARD WINNER • NE...,9780525657613,320.0,Fiction,4.0,98.0,en,2020,Maggie O'Farrell,Winner
1,Dominicana,Angie Cruz,2019-09-03,A GOOD MORNING AMERICA BOOK CLUB PICK Shortlis...,9781250205926,381.0,Fiction,4.0,18.0,en,2020,Angie Cruz,Finalist
2,"Girl, Woman, Other",Bernardine Evaristo,2019-11-05,NATIONAL BESTSELLER WINNER OF THE BOOKER PRIZE...,9780802156990,464.0,Fiction,4.5,52.0,en,2020,Bernardine Evaristo,Finalist
3,A Thousand Ships,Natalie Haynes,2019-05-02,Shortlisted for the Women's Prize for Fiction ...,9781509836222,352.0,Fiction,4.0,15.0,en,2020,Natalie Haynes,Finalist
4,The Mirror and the Light,Hilary Mantel,2022-03-29,AN INTERNATIONAL BESTSELLER WINNER OF THE WALT...,1443413747,912.0,nan,4.5,42.0,en,2020,Hilary Mantel,Finalist
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,The Leper's Companions,Julia Blackburn,2013-10-31,"To escape from her own sadness, a woman finds ...",9781448190904,224.0,Fiction,3.0,1.0,en,1999,Julia Blackburn,Shortlist
165,Visible Worlds,Marilyn Bowering,1999,Two brothers in 1930s Manitoba find themselves...,0006551130,308.0,Arctic regions,NaN,NaN,en,1999,Marilyn Bowering,Shortlist
166,The Short History of a Prince,Jane Hamilton,2010-09-01,Walter McCloud is a boy with dreams unlike mos...,9780307764072,368.0,Fiction,NaN,NaN,en,1999,Jane Hamilton,Shortlist
167,The Poisonwood Bible,Barbara Kingsolver,2008-09-04,**NOW INCLUDING THE FIRST CHAPTER OF DEMON COP...,9780571246212,633.0,Fiction,4.0,285.0,en,1999,Barbara Kingsolver,Shortlist


In [35]:
book_df.Categories.value_counts()

Categories
Fiction                              141
nan                                    5
Historical fiction                     1
Acquaintance rape                      1
Domestic fiction                       1
Biographical fiction                   1
Holocaust survivors                    1
Journalists                            1
Single women                           1
Actresses                              1
Alzheimer's disease                    1
African American men                   1
Performing Arts                        1
Belfast (Northern Ireland)             1
Book clubs (Discussion groups)         1
Donegal (Ireland)                      1
Biography & Autobiography              1
London (England)                       1
American fiction                       1
Blacks                                 1
Drama                                  1
Adopted children                       1
Sarajevo (Bosnia and Hercegovina)      1
Law                                    1
Arcti

In [34]:
# Collect topics for each book
book_dict = get_book_topics(book_df)

Error occurred for title: To be announced 14 June 2023, author: 


In [20]:

with open("assets/topics.json", "w") as outfile:
    json.dump(book_dict, outfile)

## My own Books from Goodreads export tool
Goodreads export using: https://www.goodreads.com/review/import

In [21]:
df = pd.read_csv("assets/goodreads_library_export.csv")

In [32]:
# Collect topics for my own books from OLapi
my_topics = get_book_topics(df)

In [33]:
# Writes the topics as 
with open("assets/my_topics.json", "w") as outfile:
    json.dump(my_topics, outfile)

In [29]:
pd.set_option('display.max_columns',100)
df.head()

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,Number of Pages,Year Published,Original Publication Year,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
0,6449422,"1Q84 (1Q84, #2)",Haruki Murakami,"Murakami, Haruki",NaN,"=""4103534230""","=""9784103534235""",5,4.10,Shinchosha/Tsai Fong Books,Hardcover,397.0,2009,2009.0,NaN,2023/04/28,NaN,NaN,read,NaN,NaN,NaN,1,0
1,6443834,"1Q84 Book 1 (1Q84, #1)",Haruki Murakami,"Murakami, Haruki",Murakami Haruki,"=""4103534222""","=""9784103534228""",5,4.02,Shinchosha/Tsai Fong Books,Hardcover,554.0,2009,2009.0,NaN,2023/04/28,NaN,NaN,read,NaN,NaN,NaN,1,0
2,61344814,Death of a Bookseller,Alice Slater,"Slater, Alice",NaN,"=""""","=""""",0,3.82,Hodder & Stoughton,Kindle Edition,384.0,2023,2023.0,NaN,2023/05/05,to-read,to-read (#128),to-read,NaN,NaN,NaN,0,0
3,25189315,Smoke Gets in Your Eyes & Other Lessons from t...,Caitlin Doughty,"Doughty, Caitlin",NaN,"=""0393351904""","=""9780393351903""",0,4.21,W. W. Norton & Company,Paperback,254.0,2015,2014.0,NaN,2023/05/05,to-read,to-read (#127),to-read,NaN,NaN,NaN,0,0
4,39897752,Lost in the Never Woods,Aiden Thomas,"Thomas, Aiden",NaN,"=""125031397X""","=""9781250313973""",0,3.72,Swoon Reads,Hardcover,384.0,2021,2021.0,NaN,2023/05/05,to-read,to-read (#126),to-read,NaN,NaN,NaN,0,0


In [37]:
from sklearn.metrics import jaccard_score

def calculate_similarity(topics1, topics2):
    set1 = set(topics1)
    set2 = set(topics2)
    similarity = jaccard_score(set1, set2)
    return similarity

ModuleNotFoundError: No module named 'sklearn'